In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv 
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append('../maths/') 
from baseball_stats import BasicPitching

In [ ]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_pitching = pd.read_csv('../datafiles/Pitching.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
pitching = people.copy().merge(import_pitching, on='playerID', how ='inner')
pitching['season_start'] = pitching.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
pitching['season_start'] = pd.to_datetime(pitching['season_start'], errors='coerce')
pitching['age'] = pitching.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)

pitching = pitching.drop(columns = ['birthYear', 'birthMonth', 'birthdate', 'birthDay', 'stint', 'teamID', 'lgID', 'season_start', 'BAOpp', 'ERA'])
pitching.rename(columns = {'yearID':'Years', 'IPouts':'IPO', 'BFP':'BF', 'SO':'K'}, inplace=True)

context_cols = ['playerID', 'age', 'Years']
data_cols = pitching.columns.difference(context_cols)

pitching[data_cols] = pitching[data_cols].fillna(0)
pitching = pitching[context_cols + data_cols.tolist()]

pitching = pitching.groupby(['playerID', 'age', 'Years']).sum().reset_index()

#easy adding shit up
pitching = BasicPitching.pitchingSums(pitching)


,playerID,age,Years,BB,BF,BK,CG,ER,G,GF,...,SF,SH,SHO,SV,W,WP,IP,DECI,NODE,PAB
0,aardsda01,22,2004,10,61.0,0,0,8,11,5,...,1.0,0.0,0,0,1,0,10.667,1,10,48.0
1,aardsda01,24,2006,28,225.0,0,0,24,45,9,...,3.0,1.0,0,0,3,1,53.000,3,42,192.0
2,aardsda01,25,2007,17,151.0,0,0,23,25,7,...,1.0,2.0,0,0,2,2,32.333,3,22,130.0
3,aardsda01,26,2008,35,228.0,0,0,30,47,7,...,2.0,3.0,0,0,4,3,48.667,6,41,183.0
4,aardsda01,27,2009,34,296.0,0,0,20,73,53,...,1.0,2.0,0,38,3,2,71.333,9,64,259.0
5,aardsda01,28,2010,25,202.0,0,0,19,53,43,...,1.0,7.0,0,31,0,2,49.667,6,47,167.0
6,aardsda01,30,2012,1,5.0,0,0,1,1,1,...,0.0,0.0,0,0,0,0,1.000,0,1,4.0
7,aardsda01,31,2013,19,178.0,1,0,19,43,7,...,1.0,2.0,0,0,2,1,39.667,4,39,152.0
8,aardsda01,33,2015,14,129.0,0,0,16,33,9,...,1.0,0.0,0,0,1,1,30.667,2,31,113.0
9,aasedo01,22,1977,19,373.0,0,4,32,13,0,...,3.0,2.0,2,0,6,0,92.333,8,5,348.0


In [3]:
career_df = pitching.copy()
career_df = career_df.groupby(['playerID']).agg({
    'age':'mean',
    'Years':'count',
    'BB':'sum',
    'BF':'sum',
    'BK':'sum',
    'CG':'sum',
    'ER':'sum',
    'G':'sum',
    'GF':'sum',
    'GIDP':'sum',
    'GS':'sum',
    'H':'sum',
    'HBP':'sum',
    'HR':'sum',
    'IBB':'sum',
    'IPO':'sum',
    'K':'sum',
    'L':'sum',
    'R':'sum',
    'SF':'sum',
    'SH':'sum',
    'SHO':'sum',
    'SV':'sum',
    'W':'sum',
    'WP':'sum',
    'IP':'sum',
    'DECI':'sum',
    'NODE':'sum',
    'PAB':'sum'
}).reset_index()

career_df['age'] = career_df['age'].round(0).astype(int)


In [4]:
avg162_df = career_df.copy()

stat_cols = avg162_df.columns.difference(['playerID', 'age', 'Years'])
avg162_df[stat_cols] = avg162_df[avg162_df['G'] > 0][stat_cols].div(avg162_df['G'], axis=0).mul(56).round(2)
avg162_df.insert(0, 'rowType', '162Avg')


In [5]:
norm_df = career_df.copy()
stat_cols = norm_df.columns.difference(['playerID', 'age', 'Years'])
norm_df[stat_cols] = norm_df[norm_df['IP'] > 0][stat_cols].div(norm_df['IP'], axis=0).mul(9).round(2)
norm_df.insert(0, 'rowType', 'normalized')

In [9]:
#merge all but season average and caluclate the percentages and ratios

season_df = pitching.copy()
season_df.insert(0, 'rowType', 'season')
career_df.insert(0, 'rowType', 'career')

most_pitching = pd.concat([season_df, career_df, avg162_df, norm_df], ignore_index=True)

most_pitching = BasicPitching.pitchingRatios(most_pitching)


In [ ]:
#create rows for season average
season_avg_df = most_pitching[most_pitching['rowType'] == 'season'].copy()
season_avg_df = season_avg_df.drop(['rowType'], axis=1).groupby(['playerID']).mean().round(4).reset_index()

season_avg_df['age'] = season_avg_df['age'].round(0).astype(int)
season_avg_df['Years'] = season_avg_df['Years'].round(0).astype(int)
season_avg_df.insert(0, 'rowType', 'seasAvg')

final_pitching = pd.concat([most_pitching, season_avg_df], ignore_index=True)

,rowType,playerID,age,Years,BB,BF,BK,CG,ER,G,...,OBA,BABIP,PFR,R9,BB9,BF9,WHIP,WPCT,BBK,OBPA
27670,season,maddugr01,19,1986,11.00,144.00,0.00,1.00,19.00,6.00,...,0.3360,0.3690,1.0000,5.806,3.1940,41.8060,1.7740,0.3330,0.550,0.4270
27671,season,maddugr01,20,1987,74.00,701.00,7.00,1.00,97.00,30.00,...,0.2940,0.3200,1.1240,6.418,4.2780,40.5290,1.6380,0.3000,0.733,0.4210
27672,season,maddugr01,21,1988,81.00,1047.00,6.00,9.00,88.00,34.00,...,0.2440,0.2710,0.8880,3.506,2.9280,37.8430,1.2490,0.6920,0.579,0.3390
27673,season,maddugr01,22,1989,82.00,1002.00,3.00,7.00,78.00,35.00,...,0.2490,0.2790,0.9100,3.399,3.0970,37.8380,1.2760,0.6130,0.607,0.3480
27674,season,maddugr01,23,1990,71.00,1011.00,3.00,8.00,91.00,35.00,...,0.2650,0.3020,0.9070,4.405,2.6960,38.3920,1.3210,0.5000,0.493,0.3470
27675,season,maddugr01,24,1991,66.00,1070.00,3.00,7.00,98.00,37.00,...,0.2370,0.2750,1.0040,3.867,2.2590,36.6160,1.1330,0.5770,0.333,0.3110
27676,season,maddugr01,25,1992,70.00,1061.00,0.00,9.00,65.00,35.00,...,0.2100,0.2560,1.0040,2.284,2.3510,35.6310,1.0110,0.6450,0.352,0.2970
27677,season,maddugr01,26,1993,52.00,1064.00,1.00,8.00,70.00,36.00,...,0.2320,0.2740,0.9330,2.865,1.7530,35.8650,1.0490,0.6670,0.264,0.2910
27678,season,maddugr01,27,1994,31.00,774.00,1.00,10.00,35.00,25.00,...,0.2070,0.2580,0.9260,1.960,1.3810,34.4850,0.8960,0.7270,0.199,0.2580
27679,season,maddugr01,28,1995,23.00,785.00,0.00,10.00,38.00,28.00,...,0.1970,0.2460,0.9730,1.674,0.9870,33.6960,0.8110,0.9050,0.127,0.2330


In [ ]:
load_dotenv()

db_user=os.getenv('jbbs_db_user')
db_pass=os.getenv('jbbs_db_password')
db_host=os.getenv('jbbs_db_host')
db_name=os.getenv('jbbs_db_name')

engine = create_engine(
    f'mysql+mysqlconnector://{db_user}:{db_pass}@{db_host}/{db_name}',
    echo=False,
    pool_size=5,      # Maximum number of connections in the pool
    max_overflow=0,   # Prevents creating more connections than `pool_size`
    pool_recycle=600,
    pool_pre_ping=True
)

with engine.connect() as conn:
    conn.rollback() 
    final_pitching.to_sql('player_pitching_new', con=engine, if_exists='replace', index=False, chunksize=500)
    conn.execute('create index idx_playerid on player_pitching_new(playerID);')
    conn.execute('create index idx_rowType on player_pitching_new(rowType);')
    conn.execute('create index idx_age on player_pitching_new(age);')
    conn.execute('create index idx_years on player_pitching_new(Years);')
